In [1]:
suppressMessages(library(tidyverse))
dat.dir <- "/scratch/PI/candes/ukbiobank"
tmp.dir <- "/scratch/PI/candes/ukbiobank_tmp"
chr <- 22

In [2]:
# Load list of variants
grp.file <- sprintf("%s/clumping/Radj%d/grp_chr%d.txt", tmp.dir, 50, chr)
Variants <- read_delim(grp.file, delim=" ", col_types=cols()) %>% 
                mutate(CHR=Chr, SNP=Variant, BP=Position) %>% select(CHR, SNP, BP, MAF, Group)

# Load sparse covariance matrix
corr.filename <- sprintf("%s/stats/ukb_gen_chr%s.ld", tmp.dir, chr)
LD <- read_table2(corr.filename, col_types=cols())
LD <- filter(LD, BP_A %in% Variants$BP, BP_B %in% Variants$BP)

# Load dendrogram
dend.filename <- sprintf("%s/clumping/Radj/grp_chr%s.RData", tmp.dir, chr)
load(dend.filename)

Warning message:
“Missing column names filled in: 'X9' [9]”

In [3]:
load.groups <- function(resolution) {
    groups <- cutree(Sigma.clust, k = round(0.01*resolution*nrow(Variants)))
    Variants.res <- mutate(Variants, Group = groups)
    Variants.grouped <- Variants.res %>% 
        mutate(CHR_A=CHR, CHR_B=CHR, BP_A=BP, BP_B=BP, SNP_A=SNP, SNP_B=SNP,
               Group_A=Group, Group_B=Group) %>% 
        select(-c("CHR", "SNP", "BP", "Group"))
    
    LD.groups <- LD %>% inner_join(select(Variants.grouped, CHR_A, BP_A, SNP_A, Group_A), 
                                   by=c("CHR_A","SNP_A","BP_A")) %>%
    inner_join(select(Variants.grouped, CHR_B, BP_B, SNP_B, Group_B), by=c("CHR_B","SNP_B","BP_B"))
    
    Representatives <- Variants.res %>% 
    group_by(CHR, Group) %>% 
    summarise(SNP=SNP[which.max(MAF)], BP=BP[which.max(MAF)], MAF=max(MAF))
    
    LD.new <- LD.groups %>% filter(Group_A!=Group_B) %>% gather(BP_A, BP_B, key=Dummy, value=BP) %>% arrange(BP)
    Groups <- Representatives %>% left_join(select(LD.new, BP, R2, DP)) %>% group_by(CHR, Group) %>%
        summarise(R2=max(R2), DP=max(DP))

    Groups$Resolution <- resolution
    return(Groups)
}

In [ ]:
Groups <- tibble()
for(resolution in c(75,50)) {
    Groups.res <- load.groups(resolution)
    Groups <- rbind(Groups, Groups.res)
}

Joining, by = "BP"
Joining, by = "BP"


In [1]:
Groups %>% group_by(Resolution) %>% summarise(Number=n(), R2=mean(R2), DP=mean(DP))

ERROR: Error in Groups %>% group_by(Resolution) %>% summarise(Number = n(), R2 = mean(R2), : could not find function "%>%"


In [5]:
resolution <- 10
groups <- cutree(Sigma.clust, k = round(0.01*resolution*nrow(Variants)))
    Variants.res <- mutate(Variants, Group = groups)
    Variants.grouped <- Variants.res %>% 
        mutate(CHR_A=CHR, CHR_B=CHR, BP_A=BP, BP_B=BP, SNP_A=SNP, SNP_B=SNP,
               Group_A=Group, Group_B=Group) %>% 
        select(-c("CHR", "SNP", "BP", "Group"))
    
    LD.groups <- LD %>% inner_join(select(Variants.grouped, CHR_A, BP_A, SNP_A, Group_A), 
                                   by=c("CHR_A","SNP_A","BP_A")) %>%
    inner_join(select(Variants.grouped, CHR_B, BP_B, SNP_B, Group_B), by=c("CHR_B","SNP_B","BP_B"))
    
    Representatives <- Variants.res %>% 
    group_by(CHR, Group) %>% 
    summarise(SNP=SNP[which.max(MAF)], BP=BP[which.max(MAF)], MAF=max(MAF))
    
    LD.new <- LD.groups %>% filter(Group_A!=Group_B) %>% gather(BP_A, BP_B, key=Dummy, value=BP) %>% arrange(BP)
    Groups <- Representatives %>% left_join(select(LD.new, BP, R2, DP)) %>% group_by(CHR, Group) %>%
        summarise(R2=max(R2), DP=max(DP))

Groups %>% head()

Joining, by = "BP"


CHR,Group,R2,DP
22,1,0.0884718,0.478484
22,2,0.1201630,0.927543
22,3,0.0408149,0.784342
22,4,0.1319160,0.995738
22,5,0.6357950,0.999284
22,6,0.6421150,0.994949


In [ ]:
install.packages("dplyr")